# MD5 Hasher
<i><b>Insecure Algorithm</b></i>

In [3]:
import hashlib
md5hasher = hashlib.md5(b'alice')
md5hasher.hexdigest()

'6384e2b2184bcbf58eccf10ca7a6563c'

# Hashing using SHA256 algorithm

In [3]:
import hashlib
hashlib.sha256(b"Nobody inspects the spammish repetition").hexdigest()

'031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406'

# Generate HMAC SHA256 signature

In [4]:
import hmac
import hashlib
import binascii

def create_sha256_signature(key, message):
    # Convert key argument (passed as a hex value in this example) to be of type bytes or bytearray
    byte_key = binascii.unhexlify(key)
    # Message argument (passed as a string) is a Unicode-object that must be encoded before hashing
    message = message.encode()
    return hmac.new(byte_key, message, hashlib.sha256).hexdigest().upper()

create_sha256_signature("E49756B4C8FAB4E48222A3E7F3B97CC3", "TEST STRING")


'55A891E416F480D5BE52B7985557B24A1028E4DAB79B64D0C5088F948EB3F52E'

# Using SALT

In [16]:
import os
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.backends import default_backend

# Generate a SALT
salt = os.urandom(16)

# Generate a Key with the help of this SALT
kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1, backend=default_backend())

# Generate a HASH with this Key
key = kdf.derive (b"my great password")

print(key)



# This is logic at the receiver end where he will verify if the generated HASH is correct
# Cant use the above kdf variable again as the Scrypt Library doesnt allow to re-use after it has been used once before

kdf_verify = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1, backend=default_backend())
kdf_verify.verify(b"my great password", key)
print("Success! (Exception if mismatch)")

b'y>\x9d\xd7\xc2fB\x7f\xe2\x1aA?8\x15\x0f\x88u\xe8\x8c\x12\xd6\x07y\x13\xb6\xf5\xaak \xcf\xa0\xe7'
Success! (Exception if mismatch)


The cryptography module is primarily a wrapper around a lower-level engine. For example, the module
can make use of OpenSSL as such an engine. This makes the system faster (because
computations aren’t being done in Python) and more secure (because it relies on a robust,
well-tested library). Throughout this book, we will always rely on default_backend()

The other parameters are specific to scrypt. The length parameter is how long the
key will be once the process is finished. In these examples, the password is processed
into an output of 32 bytes. The parameters r, n, and p are tuning parameters that
impact how long it will take to compute and how much memory is required. To better
protect your password, you want the process to take longer and require more memory,
preventing attackers from compromising large chunks of a database at once (every
compromise should take a long time).

Fortunately for you, recommended parameters are available. The r parameter
should be 8, and the p parameter should be 1. The n parameter can vary based on
whether you are doing something like a web site that needs to give a relatively prompt
response or something more securely stored that does not need quick responsiveness.
Either way, it must be a power of 2. For the interactive logins, 214 is recommended. For
the more sensitive files, a number as high as 220 is better

# AES ECB 

## Example #1

In [47]:
# NEVER USE: ECB is not secure!
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Alice and Bob's Shared Key
test_key = bytes.fromhex('00112233445566778899AABBCCDDEEFF')

aesCipher = Cipher(algorithms.AES(test_key), \
                    modes.ECB(), \
                    backend=default_backend())
aesEncryptor = aesCipher.encryptor()
aesDecryptor = aesCipher.decryptor()

message = b"""
FROM: FIELD AGENT ALICE
TO: FIELD AGENT BOB
RE: Meeting
DATE: 2001-1-1

Meet me today at the docks at 2300."""

# This logic to pad with character E, since ECB Algo needs 16 Bytes Block input
message += b"E" * (-len(message) % 16)
ciphertext = aesEncryptor.update(message)
print(ciphertext.hex())

# Decoding the Cipertext and 
aesDecryptor.update(ciphertext)

0aee9b602c574044778d4f6de3481cb90f3b683d1af60ed69396949eaf293eb244760fa0bb791339d557b43b250ac27c8430e159229e4bf5c7b39fe1fb72cfaba5c7412fda6ac67fe63093168f474913dbd386db053613be242c6059539f93da800d3ece3b12931be974f36ef5da4342


b'\nFROM: FIELD AGENT ALICE\nTO: FIELD AGENT BOB\nRE: Meeting\nDATE: 2001-1-1\n\nMeet me today at the docks at 2300.EEEE'

## Example #2

In [ ]:
ifile = "image/top_secret.bmp"
ofile = "image/top_secret__op.bmp"
with open(ifile, "rb") as reader:
    with open(ofile, "wb+") as writer:
        image_data = reader.read()
        header, body = image_data[:54], image_data[54:]
        body += b"\x00"*(16-(len(body)%16))
        writer.write(header + body)
        # writer.write(header + aesEncryptor.update(body))